In [1]:
import os
import gnupg
import keyring
from binance.spot import Spot
import numpy as np
import pandas as pd
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_checker import check_env
import gymnasium as gym
from gymnasium import spaces

In [5]:
gpg = gnupg.GPG()
encrypted_file_path = os.path.expanduser('~/Repositorios/binance_trader/.env.gpg')
with open(encrypted_file_path, 'rb') as file:
    datos = gpg.decrypt_file(file, passphrase=keyring.get_password("GPG_Passphrase", "gpg_python"))
if datos.ok:
    env_vars = dict(line.decode('utf-8').split('=', 1) for line in datos.data.splitlines())
    api_key = env_vars.get('API_KEY')
    secret_key = env_vars.get('SECRET_KEY')
# Inicializa el cliente de Binance
client = Spot(api_key, secret_key)

In [10]:
# Obtén información sobre el intercambio
exchange_info = client.exchange_info()
# Filtra los pares de trading contra USDT
usdc_pairs = [symbol['symbol'] for symbol in exchange_info['symbols'] if symbol['quoteAsset'] == 'USDC']

In [11]:
def get_historical_data(symbol, interval='1d'):
    klines = client.klines(symbol=symbol, interval=interval, limit=1000)
    data = []
    for kline in klines:
        data.append({
            'open_time': kline[0],
            'open': float(kline[1]),
            'high': float(kline[2]),
            'low': float(kline[3]),
            'close': float(kline[4]),
            'volume': float(kline[5]),
            'quote_asset_volume': float(kline[7]),
            'number_of_trades': kline[8],
            'taker_buy_base_asset_volume': float(kline[9]),
            'taker_buy_quote_asset_volume': float(kline[10])
        })
    return pd.DataFrame(data)

In [15]:
btcusdc_data = get_historical_data(usdc_pairs[1])
btcusdc_data.set_index('open_time', inplace=True)
btcusdc_data.index = pd.to_datetime(btcusdc_data.index, unit='ms').to_period('D')
btcusdc_data.tail()

,open,high,low,close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume
open_time,,,,,,,,,
2024-06-02,67716.00,68416.26,67256.01,67731.99,338.06009,2.292534e+07,54270,167.64465,1.136936e+07
2024-06-03,67731.98,70294.78,67576.54,68796.01,798.03092,5.514846e+07,67519,401.94912,2.777248e+07
2024-06-04,68796.00,71080.00,68549.54,70544.37,1226.98052,8.608864e+07,205914,559.56689,3.922868e+07
2024-06-05,70544.37,71771.68,70384.02,71110.36,1522.32085,1.081511e+08,63124,763.78407,5.424188e+07
2024-06-06,71114.94,71248.00,70808.11,70915.40,267.03803,1.895884e+07,17274,138.79122,9.853027e+06
